This is my project for computational intelligence based on kaggle dataset of monkeys.

Import packages

In [56]:
import os
import cv2
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

Set data folders

In [57]:
data_folder = 'data'
training_directory = os.path.join(data_folder, 'training', 'training')
test_directory = os.path.join(data_folder, 'validation', 'validation')
labels_file = os.path.join(data_folder, 'monkey_labels.txt')

Read labels

In [58]:
labels_df = pd.read_csv(labels_file)
labels_df = labels_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
labels_df.columns = labels_df.columns.str.strip()
labels = labels_df['Common Name']
labels

0               mantled_howler
1                 patas_monkey
2                  bald_uakari
3             japanese_macaque
4               pygmy_marmoset
5        white_headed_capuchin
6             silvery_marmoset
7       common_squirrel_monkey
8    black_headed_night_monkey
9               nilgiri_langur
Name: Common Name, dtype: object

Read images from files

In [ ]:
def read_image(image, size=(32, 32)):
    return cv2.resize(image,size).flatten